In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np



# TDA libraries
from ripser import ripser
from persim import plot_diagrams


In [ ]:
import pandas as pd
import networkx as nx

chunk_size = 100000
G = nx.DiGraph()
total_rows = sum(1 for _ in open('soc-redditHyperlinks-body.tsv')) - 1
reader = pd.read_csv('soc-redditHyperlinks-body.tsv', sep='\t', comment='#', chunksize=chunk_size)
rows_processed = 0

for chunk in reader:
    for _, row in chunk.iterrows():
        src = row['SOURCE_SUBREDDIT']
        tgt = row['TARGET_SUBREDDIT']
        # Add or update edge with weight
        if G.has_edge(src, tgt):
            G[src][tgt]['weight'] += 1
        else:
            G.add_edge(src, tgt, weight=1)
    rows_processed += len(chunk)
    print(f"Processed {rows_processed}/{total_rows} rows ({rows_processed/total_rows:.2%})")

print(f"Graph construction complete. Number of nodes: {G.number_of_nodes()}, Number of edges: {G.number_of_edges()}")


In [ ]:
import numpy as np

print("Graph Summary")
print("-------------")
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

# Average degree (in and out)
avg_in_degree = np.mean([d for n, d in G.in_degree()])
avg_out_degree = np.mean([d for n, d in G.out_degree()])
print(f"Average in-degree: {avg_in_degree:.2f}")
print(f"Average out-degree: {avg_out_degree:.2f}")

# Density
print(f"Density: {nx.density(G):.6f}")

# Top 5 nodes by in-degree
top_in = sorted(G.in_degree(), key=lambda x: x[1], reverse=True)[:5]
print("Top 5 nodes by in-degree:")
for node, deg in top_in:
    print(f"  {node}: {deg}")

# Top 5 nodes by out-degree
top_out = sorted(G.out_degree(), key=lambda x: x[1], reverse=True)[:5]
print("Top 5 nodes by out-degree:")
for node, deg in top_out:
    print(f"  {node}: {deg}")

# Edge weight statistics
weights = [edata['weight'] for _, _, edata in G.edges(data=True)]
print(f"Edge weight stats: min={np.min(weights)}, max={np.max(weights)}, mean={np.mean(weights):.2f}, median={np.median(weights)}")


In [ ]:
import numpy as np
import networkx as nx
from ripser import ripser
from persim import plot_diagrams
from scipy.spatial.distance import squareform

nodes = list(G.nodes)[:200]
subG = G.subgraph(nodes)

# Compute shortest path distances (1/weight)
distance_matrix = np.full((len(nodes), len(nodes)), np.inf)
for i, u in enumerate(nodes):
    lengths = nx.single_source_dijkstra_path_length(subG, u, weight=lambda u, v, d: 1/d['weight'])
    for j, v in enumerate(nodes):
        if v in lengths:
            distance_matrix[i, j] = lengths[v]
np.fill_diagonal(distance_matrix, 0)

# Symmetrize: take min or max of (i,j) and (j,i)
# Here, we use min (you can use max if you prefer)
for i in range(len(nodes)):
    for j in range(i+1, len(nodes)):
        d = min(distance_matrix[i, j], distance_matrix[j, i])
        distance_matrix[i, j] = d
        distance_matrix[j, i] = d

# Now convert to condensed form
condensed_dist = squareform(distance_matrix)

# Run persistent homology
diagrams = ripser(distance_matrix, distance_matrix=True)['dgms']
plot_diagrams(diagrams, show=True)


In [ ]:
import numpy as np

# diagrams[1] is H1 (loops), diagrams[0] is H0 (components)
H1 = diagrams[1]
persistences = H1[:,1] - H1[:,0]
# Get indices of top k most persistent features (outliers)
k = 3  # or any number you want
outlier_indices = np.argsort(persistences)[-k:]
outlier_features = H1[outlier_indices]
print("Top persistent H1 features (birth, death, persistence):")
for i, idx in enumerate(outlier_indices):
    print(f"{i+1}: Birth={H1[idx,0]:.3f}, Death={H1[idx,1]:.3f}, Persistence={persistences[idx]:.3f}")


In [ ]:
import networkx as nx

# Get the birth value of the most persistent H1 feature
most_persistent_idx = outlier_indices[-1]
birth_value = H1[most_persistent_idx, 0]

# Threshold the distance matrix at the birth value
threshold = birth_value + 1e-8  # small epsilon to include edges at birth
adjacency = (distance_matrix <= threshold).astype(int)

# Build the thresholded undirected graph
G_thresh = nx.Graph()
for i in range(len(nodes)):
    for j in range(i+1, len(nodes)):
        if adjacency[i, j]:
            G_thresh.add_edge(nodes[i], nodes[j])

# Find all cycles in the thresholded graph
# For large graphs, you may want to use nx.cycle_basis for efficiency
cycles = nx.cycle_basis(G_thresh)

# Print the largest cycle (as a proxy for the persistent feature)
if cycles:
    print("Nodes (subreddits) in a representative cycle for the most persistent H1 feature:")
    print(cycles[0])
else:
    print("No cycles found at this threshold.")


In [ ]:
import numpy as np
import networkx as nx
import gudhi as gd

# Assume G is your weighted DiGraph and nodes is your list of subreddits
nodes = list(G.nodes)[:200]
subG = G.subgraph(nodes)

# Compute the symmetric shortest-path distance matrix
distance_matrix = np.full((len(nodes), len(nodes)), np.inf)
for i, u in enumerate(nodes):
    lengths = nx.single_source_dijkstra_path_length(subG, u, weight=lambda u, v, d: 1/d['weight'])
    for j, v in enumerate(nodes):
        if v in lengths:
            distance_matrix[i, j] = lengths[v]
np.fill_diagonal(distance_matrix, 0)
# Symmetrize
for i in range(len(nodes)):
    for j in range(i+1, len(nodes)):
        d = min(distance_matrix[i, j], distance_matrix[j, i])
        distance_matrix[i, j] = d
        distance_matrix[j, i] = d
# Replace inf/nan with large finite value
finite_distances = distance_matrix[np.isfinite(distance_matrix)]
max_dist = np.max(finite_distances)
large_value = 2 * max_dist
distance_matrix[~np.isfinite(distance_matrix)] = large_value

# Build the Rips complex
rips_complex = gd.RipsComplex(distance_matrix=distance_matrix, max_edge_length=max_dist)
simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)

# Compute persistence (no extra arguments needed)
diag = simplex_tree.persistence(homology_coeff_field=2, persistence_dim_max=True)

# Extract H1 (loops) features and their representatives
H1_loops = [(i, p) for i, p in enumerate(diag) if p[0] == 1 and p[1][1] < float('inf')]
print("Top H1 features (loops):")
for idx, (dim, (birth, death)) in H1_loops:
    print(f"Feature {idx}: Birth={birth:.3f}, Death={death:.3f}, Persistence={death-birth:.3f}")


In [ ]:
for simplex, filtration_value in simplex_tree.get_filtration():
    subreddit_names = [nodes[i] for i in simplex]
    print(subreddit_names, filtration_value)


In [ ]:
import networkx as nx

# Convert to undirected graph for clique analysis
G_undirected = G.to_undirected()

# Find all maximal cliques (each is a list of nodes)
cliques = list(nx.find_cliques(G_undirected))

# Find all connected components
components = list(nx.connected_components(G_undirected))

# For each clique, check if it is a connected component (i.e., isolated)
isolated_cliques = []
for clique in cliques:
    clique_set = set(clique)
    # Check if this clique is a connected component and not part of a larger component
    if clique_set in components and len(clique) > 1:  # Only non-trivial cliques
        isolated_cliques.append(clique)

# Print results
if isolated_cliques:
    print("Isolated maximal cliques (tightly connected groups not connected to anything else):")
    for idx, clique in enumerate(isolated_cliques, 1):
        print(f"Clique {idx} (size {len(clique)}): {clique}")
else:
    print("No isolated maximal cliques found.")


In [ ]:
import networkx as nx
import numpy as np

# Convert to undirected graph for clique analysis
G_undirected = G.to_undirected()

# Find all maximal cliques
cliques = list(nx.find_cliques(G_undirected))

def clique_strength(clique, G):
    weights = []
    for i in range(len(clique)):
        for j in range(i+1, len(clique)):
            u, v = clique[i], clique[j]
            if G.has_edge(u, v):
                w = G[u][v].get('weight', 1)
                weights.append(w)
    if weights:
        return np.mean(weights)
    else:
        return 0

# Compute strength for all cliques of size > 2
clique_strengths = []
for clique in cliques:
    if len(clique) > 2:
        strength = clique_strength(clique, G_undirected)
        clique_strengths.append((clique, strength))

# Sort by strength descending and print top 10
top_cliques = sorted(clique_strengths, key=lambda x: x[1], reverse=True)[:10]

print("Top 10 cliques by average edge weight:")
for idx, (clique, strength) in enumerate(top_cliques, 1):
    print(f"{idx}. Size: {len(clique)}, Strength: {strength:.2f}, Subreddits: {clique}")


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random

# Subsample: pick 30 random nodes
sample_size = 30
sample_nodes = random.sample(list(G.nodes), sample_size)
subG = G.subgraph(sample_nodes).to_undirected()

# Find cliques in the subsample
cliques = list(nx.find_cliques(subG))
largest_clique = max(cliques, key=len) if cliques else []

# Draw the subsample
plt.figure(figsize=(8, 6))
pos = nx.spring_layout(subG, seed=42)
nx.draw(subG, pos, node_color='lightblue', with_labels=True, node_size=300, edge_color='gray', alpha=0.7)

# Highlight the largest clique in red
if largest_clique:
    nx.draw_networkx_nodes(subG, pos, nodelist=largest_clique, node_color='red', node_size=400)
    nx.draw_networkx_edges(subG, pos, edgelist=[(u, v) for u in largest_clique for v in largest_clique if u != v and subG.has_edge(u, v)], edge_color='red', width=2)
plt.title("Random Subsample of Reddit Graph with Largest Clique Highlighted")
plt.axis('off')
plt.show()


In [ ]:
import gudhi as gd

# Build a distance matrix for the subsample
nodes_sub = list(subG.nodes)
n = len(nodes_sub)
distance_matrix = np.full((n, n), np.inf)
for i, u in enumerate(nodes_sub):
    lengths = nx.single_source_dijkstra_path_length(subG, u, weight=lambda u, v, d: 1/d.get('weight', 1))
    for j, v in enumerate(nodes_sub):
        if v in lengths:
            distance_matrix[i, j] = lengths[v]
np.fill_diagonal(distance_matrix, 0)
for i in range(n):
    for j in range(i+1, n):
        d = min(distance_matrix[i, j], distance_matrix[j, i])
        distance_matrix[i, j] = d
        distance_matrix[j, i] = d
finite_distances = distance_matrix[np.isfinite(distance_matrix)]
max_dist = np.max(finite_distances)
large_value = 2 * max_dist
distance_matrix[~np.isfinite(distance_matrix)] = large_value

# Build Rips complex and simplex tree
rips_complex = gd.RipsComplex(distance_matrix=distance_matrix, max_edge_length=max_dist)
simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)

# Extract edges and triangles
edges = []
triangles = []
for simplex, filtration in simplex_tree.get_filtration():
    if len(simplex) == 2:
        edges.append(tuple(simplex))
    elif len(simplex) == 3:
        triangles.append(tuple(simplex))

# Map indices to subreddit names
edges_named = [(nodes_sub[i], nodes_sub[j]) for i, j in edges]
triangles_named = [(nodes_sub[i], nodes_sub[j], nodes_sub[k]) for i, j, k in triangles]

# Visualize
G_simplex = nx.Graph()
G_simplex.add_edges_from(edges_named)
plt.figure(figsize=(8, 6))
pos = nx.spring_layout(G_simplex, seed=42)
nx.draw_networkx_edges(G_simplex, pos, alpha=0.3)
nx.draw_networkx_nodes(G_simplex, pos, node_size=50, alpha=0.7)
for triangle in triangles_named:
    pts = [pos[n] for n in triangle]
    polygon = plt.Polygon(pts, closed=True, fill=True, color='orange', alpha=0.2)
    plt.gca().add_patch(polygon)
plt.title("Simplex Tree (1-skeleton and triangles) for Subsample")
plt.axis('off')
plt.show()


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import gudhi as gd

# 1. Larger subsample
sample_size = 100
sample_nodes = random.sample(list(G.nodes), sample_size)
subG = G.subgraph(sample_nodes).to_undirected()

# 2. Build distance matrix
nodes_sub = list(subG.nodes)
n = len(nodes_sub)
distance_matrix = np.full((n, n), np.inf)
for i, u in enumerate(nodes_sub):
    lengths = nx.single_source_dijkstra_path_length(subG, u, weight=lambda u, v, d: 1/d.get('weight', 1))
    for j, v in enumerate(nodes_sub):
        if v in lengths:
            distance_matrix[i, j] = lengths[v]
np.fill_diagonal(distance_matrix, 0)
for i in range(n):
    for j in range(i+1, n):
        d = min(distance_matrix[i, j], distance_matrix[j, i])
        distance_matrix[i, j] = d
        distance_matrix[j, i] = d
finite_distances = distance_matrix[np.isfinite(distance_matrix)]
max_dist = np.max(finite_distances)
large_value = 2 * max_dist
distance_matrix[~np.isfinite(distance_matrix)] = large_value

# 3. Use a much larger filtration threshold
rips_complex = gd.RipsComplex(distance_matrix=distance_matrix, max_edge_length=10 * max_dist)
simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)

# 4. Count simplices by dimension
simplex_counts = {}
for simplex, filtration in simplex_tree.get_filtration():
    dim = len(simplex) - 1
    simplex_counts[dim] = simplex_counts.get(dim, 0) + 1

print("Number of simplices by dimension:")
for dim in sorted(simplex_counts):
    print(f"Dimension {dim}: {simplex_counts[dim]}")

# 5. Extract edges and triangles for visualization
edges = []
triangles = []
for simplex, filtration in simplex_tree.get_filtration():
    if len(simplex) == 2:
        edges.append(tuple(simplex))
    elif len(simplex) == 3:
        triangles.append(tuple(simplex))

edges_named = [(nodes_sub[i], nodes_sub[j]) for i, j in edges]
triangles_named = [(nodes_sub[i], nodes_sub[j], nodes_sub[k]) for i, j, k in triangles]

G_simplex = nx.Graph()
G_simplex.add_edges_from(edges_named)
plt.figure(figsize=(12, 10))
pos = nx.spring_layout(G_simplex, seed=42)
nx.draw_networkx_edges(G_simplex, pos, alpha=0.3)
nx.draw_networkx_nodes(G_simplex, pos, node_size=50, alpha=0.7)
for triangle in triangles_named:
    pts = [pos[n] for n in triangle]
    polygon = plt.Polygon(pts, closed=True, fill=True, color='orange', alpha=0.2)
    plt.gca().add_patch(polygon)
plt.title("Detailed Simplex Tree (1-skeleton and triangles) for Larger Subsample and Higher Threshold")
plt.axis('off')
plt.show()


In [ ]:
import networkx as nx

G_undirected = G.to_undirected()
cliques = list(nx.find_cliques(G_undirected))

results = []
for clique in cliques:
    if len(clique) > 2:
        in_count = 0
        out_count = 0
        for node in clique:
            in_edges = [src for src in G.predecessors(node) if src not in clique]
            in_count += len(in_edges)
            out_edges = [tgt for tgt in G.successors(node) if tgt not in clique]
            out_count += len(out_edges)
        results.append({
            'clique': clique,
            'in_count': in_count,
            'out_count': out_count,
            'size': len(clique)
        })

sorted_results = sorted(results, key=lambda x: (-x['in_count'], x['out_count']))

print("Cliques with many incoming and few outgoing edges:")
for entry in sorted_results[:10]:
    print(f"Clique (size {entry['size']}): {entry['clique']}")
    print(f"  Incoming edges from outside: {entry['in_count']}")
    print(f"  Outgoing edges to outside: {entry['out_count']}\n")


In [ ]:
# Sort by in_count (ascending), then out_count (ascending)
sorted_insulated = sorted(results, key=lambda x: (x['in_count'], x['out_count']))

print("Top 10 most insulated cliques (fewest incoming, then fewest outgoing edges):")
for entry in sorted_insulated[:10]:
    print(f"Clique (size {entry['size']}): {entry['clique']}")
    print(f"  Incoming edges from outside: {entry['in_count']}")
    print(f"  Outgoing edges to outside: {entry['out_count']}\n")

In [ ]:
import networkx as nx

# Convert to undirected for clique finding
G_undirected = G.to_undirected()
cliques = list(nx.find_cliques(G_undirected))

diff_results = []
for clique in cliques:
    if len(clique) > 2:  # Only consider non-trivial cliques
        in_count = 0
        out_count = 0
        for node in clique:
            # Incoming: from outside to inside
            in_edges = [src for src in G.predecessors(node) if src not in clique]
            in_count += len(in_edges)
            # Outgoing: from inside to outside
            out_edges = [tgt for tgt in G.successors(node) if tgt not in clique]
            out_count += len(out_edges)
        diff = in_count - out_count
        diff_results.append({
            'clique': clique,
            'in_count': in_count,
            'out_count': out_count,
            'diff': diff,
            'size': len(clique)
        })

# Sort by greatest difference (in_count - out_count), descending
sorted_diff = sorted(diff_results, key=lambda x: x['diff'], reverse=True)

print("Top 10 cliques by greatest difference between incoming and outgoing edges (in - out):")
for entry in sorted_diff[:10]:
    print(f"Clique (size {entry['size']}): {entry['clique']}")
    print(f"  Incoming edges from outside: {entry['in_count']}")
    print(f"  Outgoing edges to outside: {entry['out_count']}")
    print(f"  Difference (in - out): {entry['diff']}\n")


In [ ]:
import networkx as nx

# Set your desired maximum clique size
max_clique_size = 3  # Change this to whatever maximum size you want

G_undirected = G.to_undirected()
cliques = list(nx.find_cliques(G_undirected))

diff_results = []
for clique in cliques:
    if 2 < len(clique) <= max_clique_size:  # Only consider cliques within size range
        in_count = 0
        out_count = 0
        for node in clique:
            # Incoming: from outside to inside
            in_edges = [src for src in G.predecessors(node) if src not in clique]
            in_count += len(in_edges)
            # Outgoing: from inside to outside
            out_edges = [tgt for tgt in G.successors(node) if tgt not in clique]
            out_count += len(out_edges)
        diff = in_count - out_count
        diff_results.append({
            'clique': clique,
            'in_count': in_count,
            'out_count': out_count,
            'diff': diff,
            'size': len(clique)
        })

# Sort by greatest difference (in_count - out_count), descending
sorted_diff = sorted(diff_results, key=lambda x: x['diff'], reverse=True)

print(f"Top 10 cliques (max size {max_clique_size}) by greatest difference between incoming and outgoing edges (in - out):")
for entry in sorted_diff[:10]:
    print(f"Clique (size {entry['size']}): {entry['clique']}")
    print(f"  Incoming edges from outside: {entry['in_count']}")
    print(f"  Outgoing edges to outside: {entry['out_count']}")
    print(f"  Difference (in - out): {entry['diff']}\n")


In [ ]:
print(f"Top 10 cliques (max size {max_clique_size}) by greatest difference between incoming and outgoing edges (in - out):")
for entry in sorted_diff[:50]:
    print(f"Clique (size {entry['size']}): {entry['clique']}")
    print(f"  Incoming edges from outside: {entry['in_count']}")
    print(f"  Outgoing edges to outside: {entry['out_count']}")
    print(f"  Difference (in - out): {entry['diff']}\n")
